In [74]:
import praw
import json
import pandas as pd
from pymongo import MongoClient

In [75]:
def create_reddit_object(json_file = "reddit_config.json"):

    with open(json_file) as f:
        data = json.load(f)


    reddit = praw.Reddit(
        user_agent = data['user_agent'],
        client_id = data['client_id'],
        client_secret = data['client_secret'],
        username = data['username'],
        password = data['password']
    )
    
    return reddit

reddit = create_reddit_object()

In [86]:
name_of_subreddit = "news"

In [87]:
subred = reddit.subreddit(name_of_subreddit)

In [79]:
# new = subred.new(limit = 30)

# for i in new:
#     try:
#         content = reddit.submission(url = i.url)
#         text = content.selftext
#     except: 
#         continue
#     print(i.title, i.url, len(i.comments), text)

Dating apps in Poland. https://www.reddit.com/r/poland/comments/1beqzss/dating_apps_in_poland/ 7 Which dating apps do you recommend using in Poland?
What 40PLN gets you in Poland https://www.reddit.com/gallery/1bellmi 91 
Invoice for a UK Client https://www.reddit.com/r/poland/comments/1bekzmb/invoice_for_a_uk_client/ 4 I'm a programmer living in Poland. I did a project for a client from The Uk who has an LTD.   
In Poland I pay a lump sum tax 12% (ryczałt). 

I need to create the invoice, but I'm not sure how to do it.  
Should I add any VAT to the invoice for my UK client?  
Should I write the amount in pounds?

Another developer suggested me adding VAT 0% and write "Reverse Charge."  
I'm not sure about this since reverse charge is only for the EU.
Will he attack Poland? 😔 https://www.reddit.com/r/poland/comments/1be3bvv/will_he_attack_poland/ 22 
Site to sell clothes https://www.reddit.com/r/poland/comments/1bdrpg2/site_to_sell_clothes/ 6 I sell wholesale men's clothes. What is the

In [90]:
new = subred.hot(limit = 30)

title = []
content = []
noc = []
url = []

for i in new:
    try:    
        text1 = reddit.submission(url = i.url)
        text = text1.selftext
    except:
        continue
    title.append(i.title)
    content.append(text)
    noc.append(len(i.comments))
    url.append(i.url)

data = {
    "Title": title,
    "SubReddit": content,
    "Number of comments": noc,
    "URL": url
}

df = pd.DataFrame(data)

In [91]:
df

,Title,SubReddit,Number of comments,URL
0,"Nex Benedict died by suicide, medical examiner...",[deleted],65,https://abcnews.go.com/US/nex-benedict-died-su...
1,"Dollar Tree and Family Dollar will close 1,000...",[deleted],168,https://abcnews.go.com/Business/dollar-tree-fa...


In [92]:
client = MongoClient('localhost', 27017)
db = client['Reddit']
collection = db['SubReddits']

In [93]:
records = df.to_dict(orient = 'records')
collection.insert_many(records)

InsertManyResult([ObjectId('65f3628bb424873c5ea9e903'), ObjectId('65f3628bb424873c5ea9e904')], acknowledged=True)